In [ ]:
import secml_malware
import magic
from secml.array import CArray
import os
import sklearn
import numpy as np
import configparser
import shutil
import pandas as pd
from keras import metrics
from secml_malware.models.malconv import MalConv
from secml_malware.models.c_classifier_end2end_malware import CClassifierEnd2EndMalware, End2EndModel
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score  
net = MalConv()
net = CClassifierEnd2EndMalware(net)
net.load_pretrained_model()

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')
section = 'evaluation'
eval_type = config.getint(section, 'evaluation_type')
malware_files = config.get(section, 'malware_files')
goodware_files = config.get(section, 'goodware_files')
modified_malware = config.get(section, 'modified_malware')
full_dos = config.get(section, 'full_dos')
extend_dos = config.get(section, 'extend_dos')
content_shift = config.get(section, 'content_shift')
fgs = config.get(section, 'fgsm')
gamma = config.get(section, 'gamma')
output = config.get(section, 'mal_output')
if os.path.exists(modified_malware):
    shutil.rmtree(modified_malware)
os.makedirs(modified_malware)
if os.path.exists(output):
    shutil.rmtree(output)
os.makedirs(output)

In [ ]:

def copy_files(source_folder, destination_folder):
    files = os.listdir(source_folder)
    for file_name in files:
        source_path = os.path.join(source_folder, file_name)
        destination_path = os.path.join(destination_folder, file_name)
        shutil.copy(source_path, destination_path)
        print(f"Copied {file_name} to {destination_path}")
        
        
def cf_matric(mal_files, good_files):
    predicted = []
    labels = []
    vectorized_list = []
    for i, f in enumerate(os.listdir(mal_files)):
        path = os.path.join(mal_files, f)
        with open(path, "rb") as file_handle:
            bytez = file_handle.read()
        x = End2EndModel.bytes_to_numpy(bytez, net.get_input_max_length(),256, False)
        _, confidence = net.predict(CArray(x), True)    
        if confidence[0]>confidence[1]:
            predicted.append(0)
        else:
            predicted.append(1)
        labels.append(1)
    
     
    for i, f in enumerate(os.listdir(good_files)):
        path = os.path.join(good_files, f)
        with open(path, "rb") as file_handle:
            bytez = file_handle.read()
        x = End2EndModel.bytes_to_numpy(bytez, net.get_input_max_length(),256, False)
        _, confidence = net.predict(CArray(x), True)
        if confidence[0]>confidence[1]:
            predicted.append(0)
        else:
            predicted.append(1)
        labels.append(0)

    Classification_report = classification_report(labels, predicted)
    accuracy = accuracy_score(labels, predicted)
    conf_matrix = confusion_matrix(labels, predicted)
    print("confusion_matrix", conf_matrix)
    print("classification_report", Classification_report)
    with open(os.path.join(output,"conf_matrix.txt"), "w") as text_file:
        text_file.write('classification_report:%s' % Classification_report)
        text_file.write('\naccuracy:%s' % accuracy)
        text_file.write('\nconfusion_matrix:%s' % conf_matrix)
   

In [ ]:
if eval_type!=1:
    #copy files from original malware folder and paste into modified malware folder
    copy_files(malware_files, modified_malware)    
if eval_type==1:
    cf_matric(malware_files, goodware_files)
if eval_type==2:
    # Copy adversarial files and replace with corresponding malware files in modified folder
    print("full_dos")
    copy_files(full_dos, modified_malware)
    cf_matric(modified_malware, goodware_files)
if eval_type==3:
    copy_files(extend_dos, modified_malware)
    cf_matric(modified_malware, goodware_files)
if eval_type==4:
    copy_files(content_shift, modified_malware)
    cf_matric(modified_malware, goodware_files)
if eval_type==5:
    copy_files(fgs, modified_malware)
    cf_matric(modified_malware, goodware_files)
if eval_type==6:
    copy_files(gamma, modified_malware)
    cf_matric(modified_malware, goodware_files)   